BETTER WAY 37. 내장 타입을 여러개로 내포시키기 보다는 클래스를 합성하라.

- 파이썬 내장 딕셔너리 타입을 사용하면, 객체의 생명 주기 동안 동적인 내부 상태를 잘 유지할 수 있다.
> 동적(dynamic) : 어떤 값이 들어올지 미리 알 수 없는 식별자들을 유지해야 함

ex) 학생의 이름은 미리 알 수 없는 상황이라고 하자. 이럴때는 학생별로 미리 정의된 애트리뷰트를 사용하는 대신, 딕셔너리에 이름을 저장하는 클래스를 정의할 수 있다.
> 애트리뷰트 : 속성, column

In [1]:
class SimpleGradebook :
    def __init__(self) :
        self._grades = {}

    def add_student(self, name):
        self._grades[name] = []
    
    def report_grade(self, name, score):
        self._grades[name].append(score)
    
    def average_grade(self, name):
        grades = self._grades[name]
        return sum(grades) / len(grades)

In [2]:
book = SimpleGradebook()
book.add_student('아이작 뉴턴')
book.report_grade('아이작 뉴턴', 90)
book.report_grade('아이작 뉴턴', 95)
book.report_grade('아이작 뉴턴', 80)

print(book.average_grade('아이작 뉴턴'))

88.33333333333333


._grade 딕셔너리를 변경해서 학생 이름(키)이 다른 딕셔너리(값)로 매핑하게 하고, 이 딕셔너리가 다시 과목(키)을 성적의 리스트(값)에 매핑하게 하게 함으로써 과목별 성적 구현할 수 있다.

다음 코드는 내부 딕셔너리로 defaultdict의 인스턴스를 사용해서 과목이 없는 경우를 처리한다.

In [5]:
from collections import defaultdict

class BySubjectGradebook:
    def __init__(self):
        self._grades = {} # 외부 dict

    def add_student(self, name):
        self._grades[name] = defaultdict(list) # 내부 dict
    
    def report_grade(self, name, subject, grade):
        # _grades 딕셔너리에서 name이라는 키를 가지고 있는 값을 찾는다.
        # name이름을 가진 내부 딕셔너리를 'by_subject' 변수에 할당한다.
        by_subject = self._grades[name] 
        # 내부 딕셔너리에서 subject라는 키를 가지고 있는 값을 반환한다.
        # 만약 해당 키가 없으면, 빈 리스트를 생성하여 추가한다.
        grade_list = by_subject[subject]
        # 새로운 성적 정보 grade를 grade_list에 추가한다.
        grade_list.append(grade)
    
    def average_grade(self, name):
        by_subject = self._grades[name]
        total, count = 0, 0
        for grades in by_subject.values():
            total += sum(grades)
            count += len(grades)
        return total/count

In [7]:
book = BySubjectGradebook()
book.add_student('알버트 아인슈타인')
book.report_grade('알버트 아인슈타인', '수학', 75)
book.report_grade('알버트 아인슈타인', '수학', 65)
book.report_grade('알버트 아인슈타인', '체육', 90)
book.report_grade('알버트 아인슈타인', '체육', 95)
print(book.average_grade('알버트 아인슈타인'))

81.25


Q. 각 점수의 가중치를 함께 저장해서 중간고사와 기말고사가 다른 쪽지 시험보다 더 큰 영향을 미치게 하고싶다.

> 가장 안쪽에 있는 딕셔너리가 과목(키)을 성적의 리스트(값)으로 매핑하는 것을 (성적, 가중치) 튜플의 리스트 매핑하도록 변경하는 것

In [8]:
class WeightedGradebook :
    def __init__(self):
        self._grades = {}
    
    def add_student(self, name):
        self._grades[name] = defaultdict(list)
    
    def report_grade(self, name, subject, score, weight):
        by_subject = self._grades[name]
        grade_list = by_subject[subject]
        # 성적 리스트가 튜플 인스턴스를 저장하게 했음
        grade_list.append((score, weight))
    
    def average_grade(self, name):
        by_subject = self._grades[name]
        
        score_sum, score_count = 0, 0
        for subject, scores in by_subject.items():
            subject_avg, total_weight = 0, 0

            for score, weight in scores:
                subject_avg += score * weight
                total_weight += weight
            
            score_sum += subject_avg / total_weight
            score_count += 1
            
        return score_sum / score_count

In [10]:
book = WeightedGradebook()
book.add_student('알버트 아인슈타인')
book.report_grade('알버트 아인슈타인', '수학', 75, 0.05)
book.report_grade('알버트 아인슈타인', '수학', 65, 0.15)
book.report_grade('알버트 아인슈타인', '체육', 90, 0.80)
book.report_grade('알버트 아인슈타인', '체육', 95, 0.60)
print(book.average_grade('알버트 아인슈타인'))

79.82142857142858


_grades = {
    "Alice": {
        "Math": [(90, 0.3), (80, 0.4), (70, 0.3)],
        "Science": [(85, 0.4), (75, 0.6)],
    },
    "Bob": {
        "Math": [(95, 0.5), (85, 0.5)],
        "History": [(80, 1.0)],
    }
}

이런 형태로 저장된 것

_grades 딕셔너리는 학생의 이름을 키(key)로, 학생이 수강한 과목과 해당 과목의 성적 리스트를 값(value)으로 갖는 defaultdict(list)입니다. 이 defaultdict는 새로운 키(key)가 추가될 때마다 자동으로 빈 리스트를 할당합니다.

파이썬 내장 딕셔너리와 튜플은 사용하기 편하므로 내부에 계속 딕셔너리, 리스트, 튜플 등의 계층을 추가해가면서 코드를 사용하기 쉽다.

> 그러나, 내포 단계가 두 단계 이상이 되면 더 이상 딕셔너리, 리스트, 튜플 계층을 추가하지 말아야 한다.  

>> 다른 프로그래머들이 코드를 읽기 어려워짐 & 유지보수 어려움

#### 클래스를 활용해 리팩터링하기

- 의존 관계 트리의 맨 밑바닥을 점수를 표현하는 클래스로 옮긴다

> 그렇지만, 이런 단순한 정보를 표현하는 클래스를 따로 만들면 너무 많은 비용이 든다

>> 점수는 불변 값이기 때문에 튜플이 더 적당해 보인다.

In [14]:
grades = []
grades.append((95, 0.45))
grades.append((85, 0.55))
total = sum(score * weight for score, weight in grades)
total_weight = sum(weight for _, weight in grades)
average_grades = total / total_weight

튜플을 점점 더 길게 확장시키는 패턴은 딕셔너리를 여러단계로 내포시키는 경우와 유사  

원소가 세 개 이상인 튜플을 사용한다면 다른 접근방법 생각해봐야함

> namedtuple

>> 일반적인 튜플은 인덱스로만 값을 접근할 수 있지만, namedtuple은 키-값 쌍으로도 값을 접근할 수 있기 때문에 가독성이 좋고 코드를 작성할 때 실수를 줄일 수 있음

>> namedtuple은 선언할 때 클래스 형태로 선언. 각 필드의 이름과 데이터 타입을 정의하고, 필드 이름으로 접근이 가능합니다.

In [18]:
from collections import namedtuple

Grade = namedtuple('Grade', ('score', 'weight'))

In [22]:
class Subject:
    def __init__(self):
        self._grades = []

    def report_grade(self, score, weight):
        self._grades.append(Grade(score, weight))
    
    def average_grade(self):
        total, total_weight = 0, 0
        # 특정 과목의 평균 성적 계산
        for grade in self._grades:
            total += grade.score * grade.weight
            total_weight += grade.weight
        return total / total_weight
    
class Student:
    def __init__(self):
        # _subject 라는 딕셔너리를 생성하고
        # Subject 클래스의 인스턴스를 이 딕셔너리 값으로 설정
        self._subject = defaultdict(Subject)
    
    def get_subject(self, name):
        return self._subject[name]
    
    def average_grade(self):
        total, count = 0, 0
        for subject in self._subject.values():
            # 모든 과목의 평균 성적 계산
            total += subject.average_grade()
            count += 1
        return total/count
    
# 모든 학생을 저장하는 컨테이너
# 이름을 사용해 동적으로 학생을 저장
class Gradebook:
    def __init__(self):
        # _student라는 딕셔너리를 생성하고
        # Student 클래스의 인스턴스를 이 딕셔너리 값으로 설정
        self._student = defaultdict(Student)

    def get_student(self, name):
        # name 인자를 받아 _student 딕셔너리에서 해당 이름의 학생 정보를 가져옴
        # _student 딕셔너리 : 학생 이름을 키로 가짐
        # Student 클래스의 인스턴스를 값으로 가짐
        return self._student[name]

In [24]:
book = Gradebook()
albert = book.get_student('알버트 아인슈타인')
math = albert.get_subject('수학')
math.report_grade(75, 0.05)
math.report_grade(65, 0.15)
math.report_grade(70, 0.80)
gym = albert.get_subject('체육')
gym.report_grade(100, 0.40)
gym.report_grade(85, 0.60)
print(albert.average_grade())
print(gym.average_grade())

80.25
91.0
